<a href="https://colab.research.google.com/github/tarabelo/2025-Curso-UEX/blob/main/Algoritmos%20de%20Deutsch%20y%20Deutsch-Jozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit qiskit-aer pylatexenc

In [ ]:
import numpy as np
from math import sqrt

# importing Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
# import basic plot tools
from qiskit.visualization import plot_histogram

# Funciones auxiliares

# Función para simular y mostrar el vector de estado
def obten_estado(qcirc, etiqueta="|\psi\!\!> = ", bloch=False):
    # Usamos el simulador de vector de estado
    # https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html
    sim = AerSimulator(method='statevector')
    qc_obj = transpile(qcirc, sim)
    result = sim.run(qc_obj).result()
    estado = result.get_statevector()
    display(estado.draw('latex', prefix=etiqueta))
    if bloch:
      display(estado.draw('bloch'))

# Funcion para obtener y mostrar la matriz unitaria
def obten_unitaria(qcirc, etiqueta):
    # Usamos el simulador de matriz unitaria
    sim_u = AerSimulator(method='unitary')
    qc_obj = transpile(qcirc, sim_u)
    result = sim_u.run(qcirc).result()
    unitary = result.get_unitary(qcirc)
    display(unitary.draw('latex', prefix=etiqueta))

In [ ]:
# Oráculos creados en el apartado anterior
# Oráculo para f0
def f0_oraculo():
    """
    Define un oraculo para la funcion f0
        return: circuito de 2 cúbit en forma de puerta
    """
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)

    # Aplica CNOT, primer parámetro target, segundo control
    qc.cx(x,y)

    oraculo = qc.to_gate()
    oraculo.name = "$U_{f_0}$"
    return(oraculo)

# Oráculo para f1
def f1_oraculo():
    """
    Define un oraculo para la funcion f1
        return: circuito de 2 cúbit en forma de puerta
    """
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)

    # Invierte el cúbit x
    qc.x(x)
    # Aplica CNOT, primer parámetro control, segundo target
    qc.cx(x,y)
    # recupera el cúbit x
    qc.x(x)

    oraculo = qc.to_gate()
    oraculo.name = "$U_{f_1}$"
    return(oraculo)

# Oráculo para f2
def f2_oraculo():
    """
    Define un oraculo para la funcion f2
        return: circuito de 2 cúbit en forma de puerta
    """
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)

    # Función identidad

    oraculo = qc.to_gate()
    oraculo.name = "$U_{f_2}$"
    return(oraculo)

# Oráculo para f3
def f3_oraculo():
    """
    Define un oraculo para la funcion f3
        return: circuito de 2 cúbit en forma de puerta
    """
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)

    # Invierte el cúbit y
    qc.x(y)

    oraculo = qc.to_gate()
    oraculo.name = "$U_{f_3}$"
    return(oraculo)

### Contenidos

1. [Algoritmo de Deutsch](#deutsch)
3. [Algoritmo de Deutsch-Jozsa](#dj)



-------------------------------
-------------------------------
-------------------------------

# Algoritmo de Deutsch <a name="deutsch"></a>

Las funciones $f(x): \{0,1\}\rightarrow\{0,1\}$ se denominan:

- Constantes: si $f(0)=f(1)$
- Balanceadas: si $f(0)\ne f(1)$

De las 4 funciones anteriores, $f_0$ y $f_1$ son balanceadas mientras que $f_2$ y $f_3$ son constantes.

Dada una función $f(x): \{0,1\}\rightarrow\{0,1\}$ desconocida, un algoritmo clásico necesita 2 llamadas a $f$ para determinar si es constante o balanceada.

El **algoritmo cuántico de Deutsch** puede determinarlo con una sola llamada a $f$.

Partimos del circuito siguiente:

<center><img src="https://drive.google.com/uc?export=view&id=1npbfKEwOqZA1EOUl8JeoytuQ7e0b7v1C" alt="Deutsch" width="800"  /></center>

Estado después de la puerta X:
$$
|\psi_0\rangle = |10\rangle
$$

Estado despues de las H:
$$
|\psi_1\rangle = |-+\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}\otimes \frac{|0\rangle + |1\rangle}{\sqrt{2}} = |y\rangle \otimes|x\rangle
$$

Y se tiene que:

$$
|y\oplus f(x)\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}\oplus f(x) = \frac{|0\oplus f(x)\rangle - |1\oplus f(x)\rangle}{\sqrt{2}} =
\begin{cases}
\frac{|0\rangle - |1\rangle}{\sqrt{2}}, & \text{si } f(x) = 0\\
\frac{|1\rangle - |0\rangle}{\sqrt{2}}, & \text{si } f(x) = 1
\end{cases}
$$

por lo que podemos escribir:
$$
|y\oplus f(x)\rangle = (-1)^{f(x)}\frac{|0\rangle - |1\rangle}{\sqrt{2}}
$$

De aquí se tiene que:

$$
|\psi_2\rangle = \frac{1}{\sqrt{2}}(-1)^{f(x)} (|0\rangle - |1\rangle)\otimes |x\rangle =
\frac{1}{\sqrt{2}} (|0\rangle - |1\rangle)\otimes (-1)^{f(x)}|x\rangle
$$
Y podemos escribir:
$$
(-1)^{f(x)}|x\rangle = |(-1)^{f(x)}x\rangle =
\frac{(-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle}{\sqrt{2}}
$$

Por lo tanto, el estado $|\psi_2\rangle$ queda:
$$
|\psi_2\rangle =
\frac{|0\rangle - |1\rangle}{\sqrt{2}}\otimes
\frac{(-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle}{\sqrt{2}} =
\begin{cases}
\pm\left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right] \otimes \left[\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right], & \text{si } f(0) = f(1)\\\mbox{}\\
\pm\left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right] \otimes  \left[\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right], & \text{si } f(0) \neq f(1)
\end{cases}
$$

Al aplicar la última `H`:

$$
|\psi_3\rangle =
\begin{cases}
\pm\frac{|0\rangle - |1\rangle}{\sqrt{2}} \otimes |0\rangle, & \text{si } f(0) = f(1)\\\mbox{}\\
\pm\frac{|0\rangle - |1\rangle}{\sqrt{2}} \otimes |1\rangle, & \text{si } f(0) \neq f(1)
\end{cases}
$$

Midiendo el cúbit de arriba vamos a obtener, con total certeza, un $0$ si $f(0)=f(1)$ o un $1$ si $f(0)\ne f(1)$, con una sola evaluacion de $f(x)$.


### Ejercicio

Implementa un circuito en Qiskit para probar el algoritmo de Deutsch con los oráculos definidos para las funciones $f_{0-3}$. Comprueba que es capaz de determinar si la función es constante o balanceada.

In [ ]:
# Empieza creando un circuito a partir de un oráculo con un bit de medida
def deutsch_qc(oraculo):
    x = QuantumRegister(1, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    meas = ClassicalRegister(1, name="Medida")
    qc = QuantumCircuit(x,y,meas)

    # TODO: Añade las puertas iniciales


    # TODO: Añade el oraculo


    # TODO: Añade la última puerta H


    # TODO: Añade la medida


    # Devuelve el circuito
    return qc

In [ ]:
# Probamos con f0
qc = deutsch_qc(f0_oraculo)
qc.draw('mpl')

In [ ]:
# Hacemos la simulacion
sim = AerSimulator(method='automatic')
qc_obj = transpile(qc, sim)
result = sim.run(qc_obj, shots=1024).result()
plot_histogram(result.get_counts())

In [ ]:
# TODO: Probar con las otras 3 funciones
sim = AerSimulator(method='automatic')
for f in [f1_oraculo, f2_oraculo, f3_oraculo]:
    qc = deutsch_qc(f)
    # hacemos la simulacion
    qc_obj = transpile(qc, sim)
    result = sim.run(qc_obj, shots=1024).result()
    display(plot_histogram(result.get_counts()))



---



---



---



# Algoritmo de Deutsch-Jozsa <a name="dj"></a>

Es una generalización a $n$ bits del anterior. Dada una función (oráculo) $f(\{x_{n-1},x_{n-2},\ldots,x_1,x_{0}\}) \rightarrow \{0,1\}$, que es, o bien, _balanceada_, o bien, _constante_, determinar el tipo:

- Función constante: la salida de $f(x)$ es, o bien, 0, o bien, 1 $\forall x$
- Función balanceada: devuelve 0 para la mitad de las entradas y 1 para la otra mitad

**Solución clásica**

Un algoritmo clásico necesita efectuar en el mejor caso $2$ y en el peor $2^{n-1}+1$ evaluaciones de $f(x)$, por lo que su complejidad es exponencial $\mathcal{O}(2^n)$.

**Solución cuántica**

El algoritmo cuántico ([D. Deutsch and R. Jozsa, 1992](https://doi.org/10.1098%2Frspa.1992.0167)) solo necesita una evaluación del oráculo $f(x)$ (ganancia exponencial).

El circuito usado es:

<center><img src="https://drive.google.com/uc?export=view&id=1SRf5wmYaN4NmLgjT7MXm0v4XsUEIASQW" alt="Deutsch-Jozsa" width="800"  /></center>

Pasos:

<ol>
<li>
Estado inicial:
        

$$\vert \psi_0 \rangle = \vert 1\rangle \vert0\rangle^{\otimes n} $$
</li>
    
<li>
Después de aplicar las primeras puertas H:
$$\vert \psi_1 \rangle = \frac{1}{\sqrt{2}}\left(|0\rangle - |1 \rangle \right)\otimes \frac{1}{\sqrt{2^{n}}}\sum_{x=0}^{2^n-1} \vert x\rangle =
\frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} \left(|0\rangle - |1 \rangle \right)\vert x\rangle $$
</li>
<p>
<li>
Aplicamos $U_f$ a $\vert y\rangle \vert x\rangle$ para obtener $\vert y \oplus f(x)\rangle\vert x\rangle$, recordando que $y\oplus f(x) = \tfrac{1}{\sqrt{2}} (-1)^{f(x)}(|0\rangle - |1\rangle)$:

$$\vert\psi_2\rangle  = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}(|0\rangle - |1\rangle)|x\rangle=
\frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)\otimes \frac{1}{\sqrt{2^{n}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}|x\rangle$$
          </li>
<p>
<li>Ahora aplicamos una H a cada cúbit en  $|x\rangle$ (recordemos que $H^{\otimes n}|x\rangle = \tfrac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle)$. Si ya no consideramos el cúbit de abajo, el estado es:<p>
$$
\begin{aligned}
    \lvert \psi_3 \rangle
        & = \frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)}
            \left[ \sum_{i=0}^{2^n-1}(-1)^{x \cdot i}
            \vert i \rangle \right] \\
        & = \frac{1}{2^n}\sum_{i=0}^{2^n-1}
            \left[ \sum_{x=0}^{2^n-1}(-1)^{f(x)}(-1)^{x \cdot i} \right]
            \vert i \rangle
\end{aligned}
$$
       
con $x \cdot i = x_{n-1}i_{n-1}\oplus x_{n-2}i_{n-2}\oplus \ldots \oplus x_1i_1 \oplus x_0i_0$.
   </li>

   <li>Se realiza la medida. La probabilidad de obtener todo 0 ($\vert i \rangle=  \vert 0 \rangle ^{\otimes n}$) es:
$$
       \left| \frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)} \right|^2 =
       \begin{cases}
       1, \text{si } f \text{ es constante}\\
       0, \text{si } f \text{ es balanceada}
       \end{cases}
$$
   </li>
<p>
</ol>
En conclusión, si a la salida obtenemos todo 0, $f$ es constante. Si obtenemos cualquier otro valor $f$ es balanceada.

<p></p>





<details>
    <summary><strong>Demostración de que $H^{\otimes n}|x\rangle = \tfrac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle$</strong></summary>

El uso de puertas `H` permite poner los n-cúbits en un estado de superposición.

El estado de _superposición completa_ es:

$$
|+\rangle^{\otimes n} = H^{\otimes n}|0\rangle^{\otimes n}
$$

$H^{\otimes n}$ se conoce como puerta _Walsh-Hadamard_.

Ejemplo para 4 cúbits:

$$
H^{\otimes 4}|1010\rangle = H|1\rangle\otimes H|0\rangle\otimes H|1\rangle\otimes H|0\rangle = \\
\frac{1}{4}\left[(|0\rangle - |1\rangle)\otimes (|0\rangle + |1\rangle) \otimes (|0\rangle - |1\rangle) \otimes (|0\rangle + |1\rangle)\right] = \frac{1}{4}\left[\\
|0000\rangle+|0001\rangle-|0010\rangle-|0011\rangle+\\
|0100\rangle+|0101\rangle-|0110\rangle-|0111\rangle-\\
|1000\rangle-|1001\rangle+|1010\rangle+|1011\rangle-\\
|1100\rangle-|1101\rangle+|1110\rangle+|1111\rangle\phantom{-}\right]
$$

En este ejemplo, el signo negativo aparece en los estados $|x_3x_2x_1x_0\rangle$ para los que se verifica que:
    
$$
(x_3x_2x_1x_0)\cdot(1010) = x_3\cdot 1\oplus x_2\cdot 0 \oplus x_1\cdot 1 \oplus x_0\cdot 0 = x_3\oplus x_1 = 1
$$

que son los estados para los que $x_3 \ne x_1$.
    
En general, para un estado $|x\rangle = |x_{n-1}x_{n-2}\ldots x_0\rangle$ de n-cúbits, se puede escribir:

$$
H^{\otimes n}|x\rangle = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n-1} (-1)^{x\cdot i}|i\rangle
$$

siendo $x\cdot i = x_{n-1}i_{n-1}\oplus x_{n-2}i_{n-2}\oplus \ldots \oplus x_0i_0$

</details>

**Ejemplo**
Supongamos $n=2$. Las probabilidades asociadas a cada salida son:

$$
p(0) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 0} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{0} + (-1)^{f(2)}(-1)^{0} + (-1)^{f(3)}(-1)^{0}\right]  \right|^2
$$
$$
p(1) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 1} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{1} + (-1)^{f(2)}(-1)^{0} + (-1)^{f(3)}(-1)^{1}\right]  \right|^2
$$
$$
p(2) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 2} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{0} + (-1)^{f(2)}(-1)^{1} + (-1)^{f(3)}(-1)^{1}\right]  \right|^2
$$
$$
p(3) = \left|\frac{1}{4} \sum_{x=0}^{3}(-1)^{f(x)}(-1)^{x \cdot 3} \right|^2 = \left|\frac{1}{4} \left[ (-1)^{f(0)}(-1)^{0} + (-1)^{f(1)}(-1)^{1} + (-1)^{f(2)}(-1)^{1} + (-1)^{f(3)}(-1)^{0}\right]  \right|^2
$$

Es fácil ver que si $f$ es constante, $p(0)=1$ y $p(1)=p(2)=p(3)=0$, ya que se anulan los términos. Por contra, si $f$ es balanceada $p(0)=0$.

### Ejemplo en Qiskit

**1. Implementamos los oráculos constante y balanceado**

In [ ]:
def cte_oraculo(n):
    """
    Define un oraculo para una función constante de n bits
        return: circuito de n+1 cúbits en forma de puerta
    """
    qc = QuantumCircuit(n+1)

    # En un oraculo cte. f(x) puede ser o siempre 0 o siempre 1,
    # lo elegimos random
    salida = np.random.randint(2)
    if salida == 1:
        qc.x(n) # Ponemos a 1 el cubit ancilla

    # Salida
    return qc

def bal_oraculo(n):
    """
    Define un oraculo para una función balanceada de n bits
        return: circuito de n+1 cúbits en forma de puerta
    """
    qc = QuantumCircuit(n+1)

    # Ejemplo con CNOTS: f(x) = 0 si x tiene paridad par (nº de bits 1 par) o 1 en otro caso.
    for q in range(n):
        qc.cx(q,n)

    # Salida
    return qc

def dj_oraculo(n, tipo):
    """
    Devuelve un oraculo para el algoritmo de Deutsch-Jozsa
        n: Número de cúbits de entrada
        tipo: tipo de oráculo, 0 cte., 1 balanceado
        return: circuito de n+1 cúbit en forma de puerta
    """
    x = QuantumRegister(n, name="|x\\rangle")
    y = QuantumRegister(1, name="|y\\rangle")
    qc = QuantumCircuit(x,y)

    if tipo == 0:          # Oraculo cte.
        qc = cte_oraculo(n)
    else:                  # Oraculo bal.
        qc = bal_oraculo(n)

    # Salida
    oraculo = qc.to_gate()
    oraculo.name = "$U_{f}$"
    return oraculo

**2. Implementamos paso a paso el circuito completo del algoritmo de Deutsch-Jozsa:**

Paso 0: Obtén el estado inicial:
        
$$\vert \psi_0 \rangle = \vert 1\rangle\vert0\rangle^{\otimes n} $$

In [ ]:
def paso_0(n):
    qc = QuantumCircuit(n+1,n)

    # TODO: Invierre el cúbit más significativo


    return qc

# Mostramos el circuito
n = 4
paso_0(n).draw('mpl')

Paso 1: Obtén el estado
$$\vert \psi_1 \rangle = \frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} \left(|0\rangle - |1 \rangle \right)\vert x\rangle $$

In [ ]:
def paso_1(n):
    qc = paso_0(n)

    # TODO: Añade las puertas H para obtener el estado 𝜓1


    return qc

# Mostramos el circuito
paso_1(n).draw('mpl')

Paso 2: Añadimos el oráculo para obtener:
$$
\lvert \psi_2 \rangle  
                 = \frac{|0\rangle - |1\rangle}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1}(-1)^{f(x)}|x\rangle  
$$

In [ ]:
def paso_2(n, tipo_oraculo):
    qc = paso_1(n)

    # Añadimos barreras para una mejor visualización
    qc.barrier()

    # TODO: Añade el oráculo
    qc =

    # Añadimos barreras para una mejor visualización
    qc.barrier()

    return qc

# Mostramos el circuito
tipo_oraculo = np.random.randint(2)

paso_2(n, tipo_oraculo).draw('mpl')

Paso 3: Añade las últimas puertas H y la medida de los $n$ primeros cúbits.

In [ ]:
def paso_3(n, tipo_oraculo):
    qc = paso_2(n, tipo_oraculo)

    # TODO: Añade las puertas H y la medida


    return qc

# Selecciona un oráculo aleatorio
tipo_oraculo = np.random.randint(2)

dj_circuito = paso_3(n, tipo_oraculo)
dj_circuito.draw('mpl')

Por último, simulamos el circuito y obtenemos un histograma de la salida:

In [ ]:
# Usamos Aer
sim = AerSimulator(method='automatic')
qc_obj = transpile(dj_circuito, sim)
result = sim.run(qc_obj, shots=1024).result()
salida = result.get_counts()
plot_histogram(salida)

Verifica que si el oráculo es constante, la salida es 0, y si es balanceado es $\ne 0$.

-------------------------------
-------------------------------
-------------------------------